In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="1AwUmDPCaKw7TZWLKwrSQ2n9z",
    project_name="ae4-unsupervised-notebook",
    workspace="jean-lucvanzyl",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/jean-lucvanzyl/ae4-unsupervised-notebook/079b1fbcb2b8411493a30ca0bf8bbfbc



In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp


# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

In [4]:
#Movies dataframe
movies_df = pd.read_csv('movies.csv')

#Genome_scores dataframe
gs_df = pd.read_csv('genome_scores.csv')

#Genome_tags dataframe
gt_df = pd.read_csv('genome_tags.csv')

#imbd dataframe
imdb_df = pd.read_csv('imdb_data.csv')

#links dataframe
links_df = pd.read_csv('links.csv')

#tags dataframe
tags_df = pd.read_csv('tags.csv')

#train dataframe
train_df = pd.read_csv('train.csv')

#test dataframe 
test_df = pd.read_csv('test.csv')

#sample submission
sample_df = pd.read_csv('sample_submission.csv')

In [5]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


df = pd.DataFrame(train_df)
del df['timestamp']


# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(0, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)# The columns must correspond to user id, item id and ratings (in that order).

In [10]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.000001, shuffle = False, random_state = 42)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8224


0.8224046892013978

In [13]:
test_df.loc[:, 'rating'] = 0

In [14]:
test_df.head()

,userId,movieId,rating
0,1,2011,0
1,1,4144,0
2,1,5767,0
3,1,6711,0
4,1,7318,0


In [15]:
test_processed = Dataset.load_from_df(test_df[['userId','movieId','rating']], reader)

In [16]:
NA, test = train_test_split(test_processed, test_size=1.0, shuffle = False, random_state = 42)

In [17]:
predictions = algo.test(test)

In [18]:
est = [i.est for i in predictions]

In [19]:
est

[3.1101551033859867,
 4.30070690069197,
 3.816113083597001,
 4.227606670357417,
 3.1362720705582654,
 3.9140856210604484,
 4.2790387679484425,
 3.6281627737903737,
 3.7452118900079023,
 3.3509064270349795,
 3.542121247927632,
 3.1430372930600963,
 3.859903280049138,
 4.496554406783572,
 3.7398029101028776,
 3.8072789374886082,
 3.6339076960066063,
 2.732103360068935,
 3.5448751213775775,
 3.332865004355619,
 3.9659026235314876,
 3.2624791108511193,
 3.208896931835637,
 3.131387680438112,
 3.619112757419265,
 3.306557433642059,
 3.57961710592333,
 3.1855403662701267,
 3.7053469045976213,
 3.4350339450386724,
 3.9273943016054087,
 3.1663506682165536,
 3.1584714416974946,
 2.928277400045819,
 2.566763354179659,
 3.4616427328552,
 3.483313208304001,
 3.6405704207859952,
 5,
 3.4679425804840656,
 3.8732057063754795,
 3.6173418415846896,
 4.864705240452382,
 3.3984204761326327,
 4.457582959852191,
 3.66005018224722,
 4.202477309161379,
 4.594856485913056,
 4.0139947533429154,
 4.423182571230

In [20]:
test_df.head()

,userId,movieId,rating
0,1,2011,0
1,1,4144,0
2,1,5767,0
3,1,6711,0
4,1,7318,0


In [21]:
del test_df['rating']

In [42]:
test_df['rating'] = est

In [23]:
test_df.head()

,userId,movieId,rating
0,1,2011,3.110155
1,1,4144,4.300707
2,1,5767,3.816113
3,1,6711,4.227607
4,1,7318,3.136272


In [24]:
test_df['Id'] = test_df['userId'].astype(str)  + '_'+ test_df['movieId'].astype(str) 

In [25]:
test_df.head()

,userId,movieId,rating,Id
0,1,2011,3.110155,1_2011
1,1,4144,4.300707,1_4144
2,1,5767,3.816113,1_5767
3,1,6711,4.227607,1_6711
4,1,7318,3.136272,1_7318


In [26]:
del test_df['userId']
del test_df['movieId']
test_df['rating'] = est

In [27]:
test_df.head()

,rating,Id
0,3.110155,1_2011
1,4.300707,1_4144
2,3.816113,1_5767
3,4.227607,1_6711
4,3.136272,1_7318


In [33]:
df_reset= test_df.reset_index(drop=True)

In [34]:
df_reset

,rating,Id
0,3.110155,1_2011
1,4.300707,1_4144
2,3.816113,1_5767
3,4.227607,1_6711
4,3.136272,1_7318
...,...,...
5000014,3.426744,162541_4079
5000015,4.030576,162541_4467
5000016,2.749816,162541_4980
5000017,2.911957,162541_5689


In [35]:
test_df.reset_index(drop=True, inplace=True)

In [37]:
test_df

,rating,Id
0,3.110155,1_2011
1,4.300707,1_4144
2,3.816113,1_5767
3,4.227607,1_6711
4,3.136272,1_7318
...,...,...
5000014,3.426744,162541_4079
5000015,4.030576,162541_4467
5000016,2.749816,162541_4980
5000017,2.911957,162541_5689


In [43]:
test_df.head()

,Id,rating
0,1_2011,3.110155
1,1_4144,4.300707
2,1_5767,3.816113
3,1_6711,4.227607
4,1_7318,3.136272


In [46]:
test_df.to_csv('RMSE_0.81.csv', index = False)

In [47]:
rounded_df = test_df

In [48]:
def custom_round(x, base=5):
    return base * round(float(x)/base)

rounded_df['rating'] = rounded_df['rating'].apply(lambda x: custom_round(x, base=.5))

In [50]:
rounded_df.to_csv('RMSE_0.81_rounded.csv', index = False)

In [ ]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="1AwUmDPCaKw7TZWLKwrSQ2n9z",
    project_name="ae4-unsupervised-notebook",
    workspace="jean-lucvanzyl",
)

In [29]:
movies_df.head(100)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
95,97,"Hate (Haine, La) (1995)",Crime|Drama
96,98,Shopping (1994),Action|Thriller
97,99,Heidi Fleiss: Hollywood Madam (1995),Documentary
98,100,City Hall (1996),Drama|Thriller


In [31]:
train_df.shape

(10000038, 3)

In [32]:
train_df.head()

,userId,movieId,rating
0,5163,57669,4.0
1,106343,5,4.5
2,146790,5459,5.0
3,106362,32296,2.0
4,9041,366,3.0


In [45]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/jean-lucvanzyl/ae4-unsupervised-notebook/079b1fbcb2b8411493a30ca0bf8bbfbc
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (11 MB)
COMET INFO:     installed packages       : 1
COMET INFO:     notebook                 : 1
COMET INFO:     source_code              : 1
COMET INFO: ---------------------------
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
COMET INFO: Waiting for completion of the file uploads (may take several seconds)
COMET INFO: The Python SDK ha